In [ ]:
import requests
from kafka import KafkaProducer
import json

def send_thingspeak_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from ThingSpeak
    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        feeds = data.get("feeds", [])

        # Publish each entry to Kafka
        for feed in feeds:
            for field in fields:
                feed[fields[field]] = feed.pop(field)
            producer.send(topic, value=feed)
            #print(f"Sent: {feed}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(feeds)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [2]:
import requests
response = requests.get("https://opensky-network.org/api/states/all")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en dictionnaire
    
    # Vérifier si "states" existe dans les données
    if "states" in data and data["states"]:
        for state in data["states"]:
            icao24 = state[0]  # Identifiant unique de l'avion (ICAO24)
            timestamp = state[3]  # Timestamp de la dernière position
            latitude = state[6]  # Latitude de l'avion
            longitude = state[5]  # Longitude de l'avion

            # Afficher uniquement si latitude et longitude sont disponibles
            if latitude is not None and longitude is not None:
                print(f"ID: {icao24}, Timestamp: {timestamp}, Latitude: {latitude}, Longitude: {longitude}")
    else:
        print("Aucune donnée d'avion disponible.")
else:
    print("Erreur :", response.text)

ID: 4b1819, Timestamp: 1742285471, Latitude: 47.4536, Longitude: 8.5565
ID: 738a86, Timestamp: 1742285463, Latitude: 31.4007, Longitude: 34.7054
ID: 88044a, Timestamp: 1742285458, Latitude: 10.071, Longitude: 99.4153
ID: 4b1812, Timestamp: 1742285306, Latitude: 51.476, Longitude: -0.4578
ID: 88044e, Timestamp: 1742285334, Latitude: 13.926, Longitude: 10.6083
ID: 88044d, Timestamp: 1742285476, Latitude: 14.0639, Longitude: 100.4506
ID: 7c6b41, Timestamp: 1742285476, Latitude: -36.0276, Longitude: 147.3264
ID: 842194, Timestamp: 1742285473, Latitude: 35.5581, Longitude: 139.7731
ID: 440da5, Timestamp: 1742285476, Latitude: 46.2065, Longitude: 16.7636
ID: 880454, Timestamp: 1742285399, Latitude: 13.921, Longitude: 100.6068
ID: 880453, Timestamp: 1742285440, Latitude: 13.9162, Longitude: 10.604
ID: 880451, Timestamp: 1742285463, Latitude: 13.9143, Longitude: 100.6057
ID: 80162f, Timestamp: 1742285476, Latitude: 19.6821, Longitude: 72.8072
ID: 440da9, Timestamp: 1742285476, Latitude: 48.371